In [2]:
from canvasapi import Canvas, assignment
import canvasapi as cnv
from tokens import canvas_token, notion_token
from notion_client import Client
from datetime import datetime,timedelta
import pytz

In [3]:
nclient = Client(auth=notion_token)
API_URL = "https://osu.instructure.com/"
API_KEY = canvas_token
canvas = Canvas(API_URL,API_KEY)
kiran = canvas.get_current_user()
courses = []
enrollment_term_id = 164
for course in kiran.get_courses():
    if course.__dict__['enrollment_term_id'] == enrollment_term_id:
        courses.append(course)
db_id = "57c6c1fdb5b54861af2fb7787344b4fc"
subjects = {}

In [68]:
nclient.databases.retrieve(db_id)

{'object': 'database',
 'id': '57c6c1fd-b5b5-4861-af2f-b7787344b4fc',
 'cover': {'type': 'external',
  'external': {'url': 'https://www.notion.so/images/page-cover/woodcuts_3.jpg'}},
 'icon': {'type': 'emoji', 'emoji': '🎒'},
 'created_time': '2021-09-28T02:20:00.000Z',
 'created_by': {'object': 'user',
  'id': 'bcd9f059-dbe8-44c2-8252-3c94a76e3e36'},
 'last_edited_by': {'object': 'user',
  'id': 'bcd9f059-dbe8-44c2-8252-3c94a76e3e36'},
 'last_edited_time': '2023-01-23T12:06:00.000Z',
 'title': [{'type': 'text',
   'text': {'content': 'General Assignments', 'link': None},
   'annotations': {'bold': False,
    'italic': False,
    'strikethrough': False,
    'underline': False,
    'code': False,
    'color': 'default'},
   'plain_text': 'General Assignments',
   'href': None}],
 'description': [],
 'is_inline': True,
 'properties': {'Due Date': {'id': '%3CL%5B%60',
   'name': 'Due Date',
   'type': 'date',
   'date': {}},
  'Scheduled Date': {'id': 'F%3Dd%3A',
   'name': 'Scheduled Date

In [4]:
def get_schema(assignment:assignment,subject_dict:dict,course:str):
    status = ""
    if assignment.due_at_date.date()-datetime.today().date()<=timedelta(days=7):
        status = str(1)
    else:
        status = str(2)
    if course in subject_dict.keys():
        subject = subject_dict[course]
    else:
        subject = {'name':course}
    schema = {
    "Status":{"status":{"id":status}},
    'Name': {"title": [{"type":"text","text":{"content":assignment.name}}]},
    "Subject":{"select":subject},
    "Due Date":{"date":{"start":datetime.strptime(assignment.due_at,"%Y-%m-%dT%H:%M:%S%z").astimezone(pytz.timezone('US/Eastern')).strftime("%Y-%m-%dT%H:%M:%S%z")}},
    "Link":{"url":assignment.html_url},
    "Time to complete":{"select":{'id': 'b261419e-254f-44b9-8788-1532a54646fb','name': 'About an hour','color': 'red'}}
    }
    return schema

In [6]:
def add_assignment(assignment:assignment,subject_dict:dict,course:str):
    return nclient.pages.create(**{"parent":{"type":"database_id","database_id":db_id},"properties":get_schema(assignment,subjects,course)})


In [7]:
def update_course(course:cnv.course.Course,subject_dict:dict):
    course_assignments = [y for y in kiran.get_assignments(course)]
    for x in course_assignments:
        if course.name in subject_dict.keys():
            subject = subject_dict[course.name]['name']
        else:
            subject = course.name
        if len(nclient.databases.query(**{"database_id":db_id,"filter":{"and":[{"property":"Name","title":{"contains":x.name}},{"property":"Subject","select":{"equals":subject}}]}})['results'])<1 and '''x.get_submission(kiran).submitted_at==None''' and x.due_at!=None:
            print(x)
            print(add_assignment(assignment=x,subject_dict=subject_dict,course=course.name)['properties']['Name'])

In [16]:
def update_all(courses:list,subject_dict:dict):
    backlog = "Backlog"
    week_backlog = "Week Backlog"
    backloglist = nclient.databases.query(**{"database_id":db_id,"filter":{"property":"Status","status":{"equals":"Backlog"}}})['results']
    for page in backloglist:
        date = datetime.fromisoformat(page['properties']['Due Date']['date']['start'])
        if date.date()-datetime.today().date()<timedelta(days=7):
            nclient.pages.update(page['id'],**{'properties':{'Status':{'status':{'name':week_backlog}}},"sorts":[{"property":"Due Date","direction":"ascending"}]})
    for x in courses:
        update_course(x,subject_dict)

In [119]:
nclient.databases.__module__

'notion_client.api_endpoints'

In [131]:
backloglist = nclient.databases.query(**{"database_id":db_id,"filter":{"property":"Status","status":{"equals":"Backlog"}},"sorts":[{"property":"Due Date","direction":"ascending"}]})['results']

In [103]:
weekbackloglist = nclient.databases.query(**{"database_id":db_id,"filter":{"property":"Status","status":{"equals":"Week Backlog"}}})['results']

In [82]:
nclient.databases.query(**{"database_id":db_id,"filter":{"property":"Name","title":{"contains":"PD1 - Problem Definition: User Persona & Pains"}}})['results'][0]['properties']['Status']['status'] == backloglist[0]['properties']['Status']['status']

True

In [104]:
for x in weekbackloglist:
    print(x['properties']['Name']['title'][0]['plain_text'])

D03 - Empathy
Syllabus Quiz 2
Email Jim Davis


In [132]:
print(len(backloglist))

100


In [133]:
for x in backloglist:
    print(x['properties']['Name']['title'][0]['plain_text'])

PD3 - Creating Value: Gains & User Needs
Homework #9
Graphics App G02
Graphics Prep G03
Project #2
Discussion 2: To Tell the Truth
Homework #10
PD4 - Market Character
Homework #11
D04 - How do you want people to think of you?
TM3 - CATME Team Evaluation 1
PD5 - Research Results & Value Proposition
Homework #12
Graphics Prep G04 Quiz
Graphics App G03
Graphics Prep G04
Homework #13
Project #3
Homework #14
P1 - Mission and Value Proposition
D05 - Making the most of little time
TM4 - Committee Meeting 1
Problem Analysis Paper: DRAFT & Peer Review
Homework #15
Graphics App G04
Graphics Prep G05
Midterm Exam #1
Homework #16
R1 - Problem Definition Review
CD1 - Concept Brainstorming & Ideation
Project #4
Problem Analysis Paper: FINAL
Mid-Semester Instructional Team Evaluations
TM5 - Project & Teamwork Check-In 1
TM6 - Revised Team Working Agreement
CD2 - Concept Selection
Graphics Prep G06 Quiz
Graphics App G05
Graphics Prep G06
Homework #17
Homework #18
pR&D Microcontrollers 1 Prep Quiz
CD3 

In [ ]:
update_all(courses=courses,subject_dict=subjects)

In [112]:
backloglist[1]

{'object': 'page',
 'id': '1340570f-8662-486c-9717-ad4c86945fc3',
 'created_time': '2023-01-20T16:41:00.000Z',
 'last_edited_time': '2023-01-20T16:41:00.000Z',
 'created_by': {'object': 'user',
  'id': '6c910c0f-1f0f-432e-9e79-324309e6457f'},
 'last_edited_by': {'object': 'user',
  'id': '6c910c0f-1f0f-432e-9e79-324309e6457f'},
 'cover': None,
 'icon': None,
 'parent': {'type': 'database_id',
  'database_id': '57c6c1fd-b5b5-4861-af2f-b7787344b4fc'},
 'archived': False,
 'properties': {'Due Date': {'id': '%3CL%5B%60',
   'type': 'date',
   'date': {'start': '2023-03-08T11:10:00.000-05:00',
    'end': None,
    'time_zone': None}},
  'Scheduled Date': {'id': 'F%3Dd%3A', 'type': 'date', 'date': None},
  'Link': {'id': 'F%7CBr',
   'type': 'url',
   'url': 'https://osu.instructure.com/courses/139445/assignments/3264833'},
  'Project': {'id': 'N%7DUj',
   'type': 'relation',
   'relation': [],
   'has_more': False},
  'Subject': {'id': '%60QUQ',
   'type': 'select',
   'select': {'id': 'd42

In [303]:
def add_links(course:cnv.course.Course):
    course_assignments = [y for y in kiran.get_assignments(course)]
    for x in course_assignments:
        if x.get_submission(kiran).submitted_at == None and x.due_at!=None:
            print(x)
            page = nclient.databases.query(**{"database_id":db_id,"filter":{"property":"Name","title":{"contains":x.name}}})['results'][0]
            nclient.pages.update(page['id'],**{'properties':{'Link':{'url':x.html_url}}})

In [57]:
def update_times(course:cnv.course.Course):
    course_assignments = [y for y in kiran.get_assignments(course)]
    for x in course_assignments:
        if x.get_submission(kiran).submitted_at == None and x.due_at!=None:
            print(x)
            page = nclient.databases.query(**{"database_id":db_id,"filter":{"and":[{"property":"Name","title":{"contains":x.name}},{"property":"Subject","select":{"equals":subjects[course.name]['name']}}]}})['results'][0]
            nclient.pages.update(page['id'],**{'properties':{'Due Date':{"date":{"start":datetime.strptime(x.due_at,"%Y-%m-%dT%H:%M:%S%z").astimezone(pytz.timezone('US/Eastern')).strftime("%Y-%m-%dT%H:%M:%S%z")}}}})

In [40]:
def find_duplicates(course_list):
    for c in course_list:
        course_assignments = [y for y in kiran.get_assignments(c)]
        for x in course_assignments:
            subject = c.name
            if len(nclient.databases.query(**{"database_id":db_id,"filter":{"and":[{"property":"Name","title":{"contains":x.name}},{"property":"Subject","select":{"equals":subject}}]}})['results'])>0 and '''x.get_submission(kiran).submitted_at==None''' and x.due_at!=None:
                print(x)

In [53]:
def remove_duplicates(course_list):
    for c in course_list:
        course_assignments = [y for y in kiran.get_assignments(c)]
        for x in course_assignments:
            subject = c.name
            qresults = nclient.databases.query(**{"database_id":db_id,"filter":{"and":[{"property":"Name","title":{"contains":x.name}},{"property":"Subject","select":{"equals":subject}}]}})['results']
            if len(qresults)>1 and '''x.get_submission(kiran).submitted_at==None''' and x.due_at!=None:
                for x in range(len(qresults)-1):
                    currentpage = nclient.pages.update(qresults[x]['id'],**{'archived':True})